<a href="https://colab.research.google.com/github/chaeyeon-data/Label-Project/blob/main/Labels_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as _hex_pandas
import datetime as _hex_datetime
import json as _hex_json

## AI Engineering



In [28]:
!pip install OpenAI
!pip install load_dotenv

In [29]:
import requests
import os
import json
from openai import OpenAI
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Retrieve the OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")

# Check if the OpenAI API key is loaded (for testing)
print("OpenAI API Key loaded:", openai_api_key is not None)  # True if successfully loaded

def ai_engineering(ai_model, initial_instructions, business_question, training_file=""):

    messages_list=[
        {"role": "system", "content": initial_instructions},
        {"role": "user", "content": business_question}
    ]

    if training_file != "no file":
        with open(training_file, 'r') as file:
            csv_content = file.read()
        messages_list.append({"role": "system", "content": f"file: {csv_content}"})

    client = OpenAI(
            api_key="",  # this is also the default, it can be omitted
    )

    response = client.chat.completions.create(
                  model=ai_model
                , messages=messages_list
                #, stream=True
                #, temperature=0.7
                #, max_tokens=64
                #, top_p=1
    )

    bot_messages = []

    message_objs = response.choices # Access the first choice
    for message_obj in message_objs:
        message_content = message_obj.message.content # Extract the message content
        #role = stream["choices"][0]["message"]["role"]
        cleaned_json_text = message_content.strip('`json ') # clean the message content
        json_dict = json.loads(cleaned_json_text) # Parse the JSON content
        bot_messages.append(json_dict)
    #print(json_dict)

    bot_message_1 = bot_messages[0]
    return bot_message_1

OpenAI API Key loaded: False


### Label HTML



In [ ]:
def get_site_html_simple_approach(url):
    try:
        # Set custom headers with a User-Agent that mimics a browser
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }

        # Send a GET request to the provided URL with the custom headers
        response = requests.get(url, headers=headers)

        # Raise an exception if the request was unsuccessful
        response.raise_for_status()

        # Return the HTML content of the page
        return response.text
    except requests.exceptions.RequestException as e:
        # Handle any exceptions (like network issues or invalid URLs)
        return f"An error occurred: {e}"


def get_site_html_complex_approach(url):
    try:
        # Send a GET request to the provided URL
        response = requests.get(url)
        # Raise an exception if the request was unsuccessful
        response.raise_for_status()

        # Get the HTML content of the page
        site_html = response.text

        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(site_html, 'html.parser')

        # Access the <body> tag directly
        body = soup.body

        # Ensure body exists
        if body is None:
            return "No <body> tag found in the HTML."

        # Get top-level elements in the <body>
        top_level_elements = body.find_all(recursive=False)

        # Create a string to store the formatted output
        formatted_output = ""

        # Loop through each top-level element inside <body>
        for element in top_level_elements:
            formatted_output += f'Tag: {element.name}\n'
            if element.attrs:
                formatted_output += f'Attributes: {element.attrs}\n'

            # Now go one level deeper inside each top-level element
            child_elements = element.find_all(recursive=False)

            for child in child_elements:
                formatted_output += f'  Child Tag: {child.name}\n'
                if child.attrs:
                    formatted_output += f'  Child Attributes: {child.attrs}\n'
                formatted_output += '  Collapsed Content: \n'
                # Append just the first 200 characters of the child's content to simulate collapsing
                formatted_output += '  ' + child.prettify()[:2000] + '...\n\n'

        return {"site_html": site_html, "formatted_output": formatted_output}

    except requests.exceptions.RequestException as e:
        # Handle any exceptions (like network issues or invalid URLs)
        return f"An error occurred: {e}"

In [ ]:
# Example usage
url = "https://www.republicrecords.com/artists/"
print(get_site_html_simple_approach(url))


<!DOCTYPE html>
<html class="no-js" lang="en-US">

<head>
	<meta charset="UTF-8">
	<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
	<title>
		Artists | Republic Records	</title>
		
	<meta name="description" content="Official Site">

		<meta name="viewport" content="width=device-width, initial-scale=1.0">
	<link rel="profile" href="https://gmpg.org/xfn/11">

	
<style>
	@font-face {
		font-family: BauOT;
		src: url("https://www.republicrecords.com/wp-content/themes/republicrecords2_republicrecords2/assets/font-face/BauOT.otf") format("opentype");
		font-weight: 400;
		font-style: normal;
		font-display: swap;
	}
	@font-face {
		font-family: BauOT;
		src: url("https://www.republicrecords.com/wp-content/themes/republicrecords2_republicrecords2/assets/font-face/BauOT-Bold.otf") format("opentype");
		font-weight: 700;
		font-style: normal;
		font-display: swap;
	}
	@font-face {
		font-family: 'Font Awesome 5 Brands';
		src: url("https://www.republicrecords.com/wp-content/them

### Break Site HTML into smaller chunks


In [ ]:
def split_into_chunks(content, chunk_size=5000):
    # Split content into chunks of specified size
    return [content[i:i + chunk_size] for i in range(0, len(content), chunk_size)]

### GPT Call to Get Relevant HTML Chunk



In [ ]:
def agent_check_if_html_is_relevant_chunk(html_chunk, what_am_i_looking_for):
    contains_relevant_information_key = "contains_" + what_am_i_looking_for.lower().replace(" ", "_") #for example: "contains_a_list_of_artists"

    response_json = ai_engineering(
        "gpt-4o-mini"
        , """
            You are a website expert of music roster sites. Your job is to analyze a chunk of the page's HTML and tell us if this chunk represents """ + what_am_i_looking_for + """
        """
        , """
            HTML Chunk:
            """ + html_chunk + """

            You will respond in the format of ONLY ```json { "contains_relevant_information": "yes/no" }  and NO OTHER TEXT

        """
        , "no file"
    )

    # Extract the tags from the response
    contains_relevant_information = response_json["contains_relevant_information"]

    return contains_relevant_information

### Scraping HTML (with buffer of +/- 3000 characters)



In [ ]:
def find_html_chunk_with_context(site_html, html_chunk, context_size=3000):
    """
    Finds the position of the HTML chunk in the site HTML and extracts
    surrounding context characters (±context_size).

    Parameters:
        site_html (str): The complete HTML content of the site.
        html_chunk (str): The HTML chunk to locate in the site HTML.
        context_size (int): Number of characters to extract before and after the chunk.

    Returns:
        str: A section of the site HTML around the chunk, or a message if the chunk is not found.
    """
    # Clean up newlines and whitespace
    site_html = site_html.strip()
    html_chunk = html_chunk.strip()

    # Find the position of the HTML chunk
    start_pos = site_html.find(html_chunk)

    if start_pos == -1:
        return "HTML chunk not found in site HTML."

    # Determine the context positions
    start_context = max(0, start_pos - context_size)
    end_context = min(len(site_html), start_pos + len(html_chunk) + context_size)

    # Extract the content with surrounding context
    extracted_content = site_html[start_context:end_context]

    return extracted_content


### GPT Call to Find Useful Fields for Scraping



In [ ]:
def agent_find_useful_fields_for_scraping(html):
    response_json = ai_engineering(
        "gpt-4o-mini"
        , """
            Identify key information to scrape from this HTML page (record label website) about artists and outline they key fields in a sample JSON structure:
            Only give a sample json structure with a format with placeholder information.
        """
        , """
            Sample HTML:
            """ + html + """

            You will respond in the format of ONLY ```json { }  and NO OTHER TEXT

        """
        , "no file"
    )

    # Extract the tags from the response
    #list_of_tags = response_json['list_of_mood_tags_comma_separated_ordered_by_relevance']

    return response_json

### GPT Call to Write Scraping Code



In [ ]:
def agent_write_scraping_sode(html, useful_fields, url):
    response_json = ai_engineering(
        "gpt-4o-mini"
        , """
            Write python scrapong code to get the data I need:
         """
        , """
            Desired Data:
            """ + str(useful_fields) + """

            URL:
            """ + url + """

            Sample HTML:
            """ + html + """

            You will respond in the format of ONLY ```json {"python_code": ""}  and NO OTHER TEXT

        """
        , "no file"
    )

    # Extract the tags from the response
    response_code = response_json['python_code']

    return response_code

## Main Function to Get the Python Automation Code -- for Label Sites



In [ ]:
what_am_i_looking_for = "A LIST OF ARTISTS"
site_urls = [ "https://www.republicrecords.com/artists/"
, "https://interscope.com/pages/iga-artist-hub", "https://partisanrecords.com/artists", "https://concord.com/artists/" ]

for site_url in site_urls:
    print(site_url)
    site_html = get_site_html_simple_approach(site_url)
    html_chunks = split_into_chunks(site_html, 5000) # Split the HTML content into chunks of 5000 characters

    print(len(html_chunks), " HTML chunks for site: ", site_url)
    relevant_html_chunk = ""

    for html_chunk in html_chunks:
        is_relevant_html_chunk = agent_check_if_html_is_relevant_chunk(html_chunk, what_am_i_looking_for)
        if is_relevant_html_chunk == "no":
            print("Not a relevant HTML chunk")
        if is_relevant_html_chunk == "yes":
            print("CORRECT HTML chunk")
            relevant_html_chunk = html_chunk
            break


    scraping_html_with_buffer = find_html_chunk_with_context(site_html, relevant_html_chunk, context_size=5000) # Find and extract the relevant HTML code to be scraped
    print()
    #print("======================Scraping HTML:======================")
    #print(scraping_html_with_buffer)


    useful_fields_for_scraping = agent_find_useful_fields_for_scraping(scraping_html_with_buffer) # fields that need be scraped, for example: artist name, image, url etc
    print()
    print("======================Useful Fields for Scraping:======================")
    #print(useful_fields_for_scraping)

    print("======================Python Code for Scraping:======================")
    scraping_code_in_python = agent_write_scraping_sode(scraping_html_with_buffer, useful_fields_for_scraping, site_url)
    print()
    #print("======================Scraping Code:======================")
    #print(scraping_code_in_python)

    print("======================Data Scraped from Site:======================")
    try:
        # Execute the code
        scraped_data = exec(scraping_code_in_python) #Scrape Site Now
    except Exception as e:
        print(f"An error occurred: {e}")

https://www.republicrecords.com/artists/
68  HTML chunks for site:  https://www.republicrecords.com/artists/
Not a relevant HTML chunk
Not a relevant HTML chunk
Not a relevant HTML chunk
Not a relevant HTML chunk
Not a relevant HTML chunk
Not a relevant HTML chunk
Not a relevant HTML chunk
Not a relevant HTML chunk
Not a relevant HTML chunk
Not a relevant HTML chunk
Not a relevant HTML chunk
Not a relevant HTML chunk
CORRECT HTML chunk


======================Useful Fields for Scraping:======================
======================Python Code for Scraping:======================

======================Data Scraped from Site:======================
An error occurred: list index out of range
https://interscope.com/pages/iga-artist-hub
82  HTML chunks for site:  https://interscope.com/pages/iga-artist-hub
Not a relevant HTML chunk
Not a relevant HTML chunk
Not a relevant HTML chunk
Not a relevant HTML chunk
Not a relevant HTML chunk
Not a relevant HTML chunk
Not a relevant HTML chunk
Not a r

## WEIBO



In [ ]:
what_am_i_looking_for = "Artist's follower count and fan count"
site_urls = [ "https://m.weibo.cn/u/5888671022?luicode=10000011&lfid=100808686fdaf30a2ca0ff1ba6acd6aeeb280b_-_feed" ]

for site_url in site_urls:
    site_html = get_site_html_simple_approach(site_url)
    print(site_html)
    html_chunks = split_into_chunks(site_html, 5000) # Split the HTML content into chunks of 5000 characters

    print(len(html_chunks), " HTML chunks for site: ", site_url)
    relevant_html_chunk = ""

    for html_chunk in html_chunks:
        is_relevant_html_chunk = agent_check_if_html_is_relevant_chunk(html_chunk, what_am_i_looking_for)
        if is_relevant_html_chunk == "no":
            print("Not a relevant HTML chunk")
        if is_relevant_html_chunk == "yes":
            print("CORRECT HTML chunk")
            relevant_html_chunk = html_chunk
            break


    scraping_html_with_buffer = find_html_chunk_with_context(site_html, relevant_html_chunk, context_size=5000) # Find and extract the relevant HTML code to be scraped
    print()
    #print("======================Scraping HTML:======================")
    #print(scraping_html_with_buffer)


    useful_fields_for_scraping = agent_find_useful_fields_for_scraping(scraping_html_with_buffer) # fields that need be scraped, for example: artist name, image, url etc
    print()
    print("======================Useful Fields for Scraping:======================")
    print(useful_fields_for_scraping)

    print("======================Python Code for Scraping:======================")
    scraping_code_in_python = agent_write_scraping_sode(scraping_html_with_buffer, useful_fields_for_scraping, site_url)
    print()
    #print("======================Scraping Code:======================")
    #print(scraping_code_in_python)

    print("======================Data Scraped from Site:======================")
    try:
        # Execute the code
        scraped_data = exec(scraping_code_in_python) #Scrape Site Now
    except Exception as e:
        print(f"An error occurred: {e}")

<!DOCTYPE html>
<html lang="zh-cn">
<head>
    <meta charset="utf-8">
    <link rel="dns-prefetch" href="//h5.sinaimg.cn">
    <meta name="viewport" content="width=device-width,initial-scale=1,user-scalable=no,viewport-fit=cover">
    <meta name="format-detection" content="telephone=no">
    <link rel="shortcut icon" type="image/x-icon" href="/favicon.ico">
    <title>微博</title>
    <meta content="随时随地发现新鲜事！微博带你欣赏世界上每一个精彩瞬间，了解每一个幕后故事。分享你想表达的，让全世界都能听到你的心声！" name="description">
        <link rel="stylesheet" href="//h5.sinaimg.cn/marvel/v1.4.7/css/lib/base.css">
    <link rel="stylesheet" href="//h5.sinaimg.cn/marvel/v1.4.7/css/card/cards.css">
        <link rel="manifest" href="/manifest.json">
    <script>!function(e){var a,i=navigator.userAgent.toLowerCase(),n=document.documentElement,t=parseInt(n.clientWidth);if(/(iPhone|iPad|iPod|iOS)/i.test(navigator.userAgent)||i.indexOf("like mac os x")>0){var s=/os [\d._]*/gi,o=i.match(s);a=(o+"").replace(/[^0-9|_.]/gi,"").replace(/_/gi,".")}var